In [ ]:
# --- Run this whole cell in Colab ---

# ============================
# Install packages
# ============================
!pip install pyngrok flask_cors tensorflow pillow --quiet

# ============================
# Imports
# ============================
from pyngrok import ngrok
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
from tensorflow.keras.models import load_model
from PIL import Image
import threading, io, base64, os, numpy as np, time
import tensorflow as tf
from keras import config as keras_config  # for unsafe deserialization

# ============================
# CONFIG
# ============================
MODEL_PATH = "/content/cataract_full_model_stage2.keras"
LABELS = ["Normal Eye", "Cataract Eye"]
DEFAULT_PORT = 5000
NGROK_AUTHTOKEN = "YOUR_NGROK_AUTHTOKEN"  # <--- paste your own token here

# ============================
# Find free port if default is busy
# ============================
def find_free_port(default_port):
    import socket
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        s.bind(('', default_port))
        s.close()
        return default_port
    except OSError:
        s.close()
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind(('', 0))
        port = s.getsockname()[1]
        s.close()
        return port

PORT = find_free_port(DEFAULT_PORT)
print(f"✅ Using port: {PORT}")

# ============================
# Save HTML UI
# ============================
html_content = """<!DOCTYPE html>
<html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><title>Cataract Detection AI</title>
<link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap" rel="stylesheet">
<style>
body { font-family: 'Poppins', sans-serif; margin:0; padding:0; background: linear-gradient(to right, #a8edea, #fed6e3); }
header { width:100%; background-color:#27ae60; color:white; display:flex; align-items:center; justify-content:space-between; padding:10px 20px; position:fixed; top:0; left:0; z-index:100; }
header h1 { margin:0; font-size:24px; }
.container { padding:120px 25px 40px; max-width:650px; margin:0 auto; text-align:center; }
.description { color:#555; font-size:16px; margin-bottom:20px; }
.btn-container { display:flex; justify-content:space-between; max-width:450px; margin:0 auto 30px auto; gap:20px; }
.btn { flex:1; padding:25px 0; border-radius:15px; background:white; color:#27ae60; font-weight:600; font-size:18px; cursor:pointer; box-shadow:0 8px 20px rgba(0,0,0,0.2); transition:transform 0.3s, box-shadow 0.3s; text-align:center; user-select:none; }
.btn:hover { transform:scale(1.05); box-shadow:0 12px 25px rgba(0,0,0,0.3); }
#analyzeBtn { width:220px; background-color:#27ae60; color:white; border:none; border-radius:15px; padding:15px 0; font-size:18px; margin-top:20px; cursor:pointer; }
#analyzeBtn:hover { background-color:#2ecc71; }
#output { margin-top:20px; font-size:16px; color:#34495e; min-height:40px; white-space:pre-line; font-weight:bold;}
#previewImage, #webcamVideo { max-width:100%; border-radius:12px; margin-top:15px; box-shadow:0 8px 20px rgba(0,0,0,0.2); display:none; }
</style></head>
<body>
<header><h1>Cataract Detection AI</h1></header>
<div class="container">
  <div class="description">Upload an eye image or use your webcam to check for cataracts.</div>
  <div class="btn-container">
    <input type="file" id="imageUpload" accept="image/*" style="display:none;">
    <div class="btn" id="uploadBtn">Upload Image</div>
    <div class="btn" id="webcamBtn">Start Webcam</div>
  </div>
  <img id="previewImage" src="" alt="Image Preview">
  <video id="webcamVideo" autoplay playsinline></video>
  <canvas id="hiddenCanvas" style="display:none;"></canvas>
  <button id="analyzeBtn">Analyze Image</button>
  <div id="output">Ready to analyze.</div>
</div>
<script>
const outputEl = document.getElementById('output');
const analyzeBtn = document.getElementById('analyzeBtn');
const uploadBtn = document.getElementById('uploadBtn');
const webcamBtn = document.getElementById('webcamBtn');
const imageUpload = document.getElementById('imageUpload');
const previewImageEl = document.getElementById('previewImage');
const webcamVideoEl = document.getElementById('webcamVideo');
const hiddenCanvas = document.getElementById('hiddenCanvas');
let webcamStream = null;

uploadBtn.addEventListener('click', () => imageUpload.click());
imageUpload.addEventListener('change', (e) => {
    if (webcamStream) { webcamStream.getTracks().forEach(t=>t.stop()); webcamStream = null; }
    const file = e.target.files[0];
    if (file) {
        const reader = new FileReader();
        reader.onload = function(ev) {
            previewImageEl.src = ev.target.result;
            previewImageEl.style.display = 'block';
            webcamVideoEl.style.display = 'none';
            outputEl.textContent = "Image uploaded. Click 'Analyze Image' to check.";
        }
        reader.readAsDataURL(file);
    }
});

webcamBtn.addEventListener('click', async () => {
    if (webcamStream) {
        webcamStream.getTracks().forEach(track => track.stop());
        webcamStream = null;
        webcamVideoEl.style.display = 'none';
        outputEl.textContent = "Webcam stopped.";
    } else {
        try {
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            webcamVideoEl.srcObject = stream;
            webcamVideoEl.style.display = 'block';
            previewImageEl.style.display = 'none';
            outputEl.textContent = "Webcam active. Click 'Analyze Image' to analyze a frame.";
            webcamStream = stream;
        } catch (err) {
            console.error("Webcam access error:", err);
            outputEl.textContent = "Cannot access webcam.";
        }
    }
});

analyzeBtn.addEventListener('click', async function() {
    outputEl.textContent = "Analyzing...";
    let imgData;
    const canvas = hiddenCanvas;
    const context = canvas.getContext('2d');
    const TARGET_W = 224, TARGET_H = 224;

    if (previewImageEl.style.display==='block' && previewImageEl.src) {
        canvas.width = TARGET_W;
        canvas.height = TARGET_H;
        context.drawImage(previewImageEl, 0, 0, canvas.width, canvas.height);
        imgData = canvas.toDataURL('image/jpeg');
    } else if (webcamVideoEl.style.display==='block' && webcamStream) {
        canvas.width = TARGET_W;
        canvas.height = TARGET_H;
        try {
          context.drawImage(webcamVideoEl, 0, 0, canvas.width, canvas.height);
          imgData = canvas.toDataURL('image/jpeg');
        } catch (err) {
          console.error("Error capturing frame:", err);
          outputEl.textContent = "Unable to capture webcam frame.";
          return;
        }
    } else {
        outputEl.textContent = "Upload an image or start webcam first.";
        return;
    }

    try {
        const resp = await fetch('/analyze', {
            method:'POST', headers:{'Content-Type':'application/json'},
            body: JSON.stringify({image:imgData})
        });
        const result = await resp.json();
        if(result.message && result.confidence!==undefined)
            outputEl.innerHTML = `Result: <strong>${result.message}</strong><br>Confidence: ${Math.round(result.confidence*100)}%`;
        else if(result.error) outputEl.textContent = "Error: " + (result.error || JSON.stringify(result));
        else outputEl.textContent = "Unexpected response: "+JSON.stringify(result);
    } catch(e) {
        console.error(e);
        outputEl.textContent = "Prediction failed: "+e.message;
    }
});
</script>
</body></html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html_content)

# ============================
# Load Model
# ============================
model = None
if os.path.exists(MODEL_PATH):
    try:
        keras_config.enable_unsafe_deserialization()
        model = load_model(MODEL_PATH, compile=False)
        print("✅ Model loaded successfully from:", MODEL_PATH)
    except Exception as e:
        print("❌ Error loading model:", e)
else:
    print("⚠️ Model file not found at", MODEL_PATH)

# ============================
# Flask app
# ============================
app = Flask(__name__, static_folder=".", static_url_path="/")
CORS(app)

@app.route("/", methods=["GET"])
def index():
    return send_from_directory('.', 'index.html')

@app.route("/analyze", methods=["POST"])
def analyze():
    global model
    if model is None:
        return jsonify({"error":"Model not loaded"}), 500
    try:
        data = request.get_json(force=True)
        img_data = data.get("image")
        if not img_data:
            return jsonify({"error":"No image data"}), 400

        header, encoded = img_data.split(',',1)
        img_bytes = base64.b64decode(encoded)
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB").resize((224,224))
        arr = np.asarray(img).astype("float32")/255.0
        arr = np.expand_dims(arr, axis=0)

        preds = model.predict(arr, verbose=0)

        label, confidence = "Unknown", 0.0
        preds = np.array(preds)

        if preds.ndim == 2 and preds.shape[1] == 1:
            pred_val = float(preds[0][0])
            label = LABELS[1] if pred_val >= 0.5 else LABELS[0]
            confidence = pred_val if pred_val >= 0.5 else (1.0 - pred_val)
        elif preds.ndim == 1:
            if preds.size == 1:
                pred_val = float(preds[0])
                label = LABELS[1] if pred_val >= 0.5 else LABELS[0]
                confidence = pred_val if pred_val >= 0.5 else (1.0 - pred_val)
            else:
                idx = int(np.argmax(preds))
                label = LABELS[idx] if idx < len(LABELS) else f"class_{idx}"
                confidence = float(np.max(preds))
        elif preds.ndim == 2:
            idx = int(np.argmax(preds[0]))
            label = LABELS[idx] if idx < len(LABELS) else f"class_{idx}"
            confidence = float(np.max(preds[0]))
        else:
            flat = preds.ravel()
            idx = int(np.argmax(flat))
            label = LABELS[idx] if idx < len(LABELS) else f"class_{idx}"
            confidence = float(np.max(flat))

        return jsonify({"message":label, "confidence":float(confidence)})
    except Exception as e:
        print("Error in analyze:", e)
        return jsonify({"error":"internal server error","details":str(e)}),500

# ============================
# Run Flask in background
# ============================
def run_app():
    app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)

thread = threading.Thread(target=run_app)
thread.daemon = True
thread.start()

# ============================
# Ngrok tunnel
# ============================
if not NGROK_AUTHTOKEN or NGROK_AUTHTOKEN.strip()=="":
    print("⚠️ NGROK_AUTHTOKEN is empty. Skipping public URL creation.")
else:
    try:
        ngrok.set_auth_token(NGROK_AUTHTOKEN.strip())
        time.sleep(1)
        public_url = ngrok.connect(PORT).public_url
        print("🚀 ngrok public URL:", public_url)
        print("Open it in your browser to use the Cataract Detection AI.")
    except Exception as e:
        print("❌ ngrok error:", e)


Mounted at /content/drive
✅ Using port: 33659
✅ Model loaded successfully from: /content/drive/MyDrive/cataract_model/cataract_full_model_stage2.keras
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:33659
 * Running on http://172.28.0.12:33659
INFO:werkzeug:Press CTRL+C to quit


🚀 ngrok public URL: https://00e6b56e9ad0.ngrok-free.app
Open it in browser to use the AI web interface.


In [5]:
# Kill all ngrok tunnels
from pyngrok import ngrok

ngrok.kill()
print("✅ All ngrok tunnels closed.")


✅ All ngrok tunnels closed.
